In [8]:
from kafka import KafkaProducer
import requests
from bs4 import BeautifulSoup as bs
import re
from time import sleep
import json


def clean_text(text):
        cleaned_text = re.sub('[a-zA-Z]','',text)
        cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]', '', cleaned_text)
        return cleaned_text

target=["에르메스","루이비통","디올","프라다","구찌","샤넬"]
excepts=["삽니다","매입","구매","구입"]
recentSubject=""
TOPIC_NAME="test"

my_arr=[]
while True:    
    with requests.Session() as s:
        BASE_URL="https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.menuid=782&search.boardtype=L"
        res=s.get(BASE_URL)
        if res.status_code==requests.codes.ok:
            par_soup=bs(res.text, 'html.parser')
            number=par_soup.select_one("#main-area > div:nth-child(4) > table > tbody > tr:nth-child(1) > td.td_article > div.board-number > div").text
            #print(number)
            if recentSubject==number:
                print('there is no new product')
                sleep(40)
                continue
            elif recentSubject !=number:
                recentSubject=number
                print(recentSubject)
                tr_arr=par_soup.select_one("#main-area > div:nth-child(4) > table > tbody > tr")
                a_tag=tr_arr.select_one('td.td_article > div.board-list > div > a')
                map = {
                "title":a_tag.text.strip(),
                "url":'https://cafe.naver.com' + a_tag["href"],
                "brand": " "
                }
                #print(map)
                #sleep(2)
        if any(x in map["title"] for x in target) and not any(x in map["title"] for x in excepts):
            msg_split=clean_text(map["title"]).split(" ")
            for msg in msg_split:
                for tar in target:
                    val = msg.find(tar)
                    if val !=-1:
                        map['brand']=tar
                        print("pipe.py로 데이터 동기화, index:", val)
    
            brokers=["localhost:9092"]
            producer = KafkaProducer(bootstrap_servers=brokers)

            producer.send(TOPIC_NAME, json.dumps(map).encode("utf-8"))
            producer.flush()
            print(map)
            sleep(40)

945554749
pipe.py로 데이터 동기화, index: 0
{'title': '루이비통백팩 보증서 백화점정품', 'url': 'https://cafe.naver.com/ArticleRead.nhn?clubid=10050146&page=1&menuid=782&boardtype=L&articleid=945554749&referrerAllArticles=false', 'brand': '루이비통'}
945554915
there is no new product


KeyboardInterrupt: 

In [7]:
from kafka import KafkaProducer
import requests
from bs4 import BeautifulSoup as bs
import re
from time import sleep
import json

class Ine(object):
    def __init__(self):
        self.target=["에르메스","루이비통","디올","프라다","구찌","샤넬"]
        self.excepts=["삽니다","매입","구매","구입"]
        self.recentSubject=""
        self.TOPIC_NAME="test"
        self.BASE_URL="https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.menuid=782&search.boardtype=L"
        #self.producer=KafkaProducer(bootstrap_servers=brokers)
        #self.brokers=["localhost:9092"]
        
    def clean_text(self,text):
            cleaned_text = re.sub('[a-zA-Z]','',text)
            cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]', '', cleaned_text)
            return cleaned_text

    def data_collect(self):

        with requests.Session() as s:
            #BASE_URL="https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.menuid=782&search.boardtype=L"
            res=s.get(self.BASE_URL)
            if res.status_code==requests.codes.ok:
                par_soup=bs(res.text, 'html.parser')
                number=par_soup.select_one("#main-area > div:nth-child(4) > table > tbody > tr:nth-child(1) > td.td_article > div.board-number > div").text

                if self.recentSubject==number:
                    print('there is no new product')
                    sleep(40)
                elif self.recentSubject !=number:
                    self.recentSubject=number
                    print(self.recentSubject,"new number -> start preprocessing")
                    tr_arr=par_soup.select_one("#main-area > div:nth-child(4) > table > tbody > tr")
                    a_tag=tr_arr.select_one('td.td_article > div.board-list > div > a')
                    map = {
                    "title":a_tag.text.strip(),
                    "url":'https://cafe.naver.com' + a_tag["href"],
                    "brand": " "
                    }
                    #print(map)
                    #sleep(2)
                if any(x in map["title"] for x in self.target) and not any(x in map["title"] for x in self.excepts):
                    msg_split=self.clean_text(map["title"]).split(" ")
                    for msg in msg_split:
                        for tar in self.target:
                            val = msg.find(tar)
                            if val !=-1:
                                map['brand']=tar
                                print("pipe.py로 데이터 동기화, index:", val)


#                    brokers=["localhost:9092"]
#                    producer = KafkaProducer(bootstrap_servers=brokers)

#                    producer.send(self.TOPIC_NAME, json.dumps(map).encode("utf-8"))
#                    producer.flush()
                    print(map)
                    sleep(40)
                            

if __name__ == '__main__':
    while True:
        vo = Ine()
        print(vo.data_collect())
        
        
        
    
# brokers=["localhost:9092"]
# producer = KafkaProducer(bootstrap_servers=brokers)

# producer.send(self.TOPIC_NAME, json.dumps(map).encode("utf-8"))
# producer.flush()
# print(map)
# sleep(40)


945553563 new number -> start preprocessing
pipe.py로 데이터 동기화, index: 0
{'title': '[정품} 구찌 인터로킹G 미니토트 크로스백', 'url': 'https://cafe.naver.com/ArticleRead.nhn?clubid=10050146&page=1&menuid=782&boardtype=L&articleid=945553563&referrerAllArticles=false', 'brand': '구찌'}
None
945553737 new number -> start preprocessing
pipe.py로 데이터 동기화, index: 0
{'title': '루이비통 페이보릿mm,', 'url': 'https://cafe.naver.com/ArticleRead.nhn?clubid=10050146&page=1&menuid=782&boardtype=L&articleid=945553737&referrerAllArticles=false', 'brand': '루이비통'}
None
945553827 new number -> start preprocessing
pipe.py로 데이터 동기화, index: 0
{'title': '[정품] 루이비통 클루니bb 에삐', 'url': 'https://cafe.naver.com/ArticleRead.nhn?clubid=10050146&page=1&menuid=782&boardtype=L&articleid=945553827&referrerAllArticles=false', 'brand': '루이비통'}
None
945553945 new number -> start preprocessing
pipe.py로 데이터 동기화, index: 0
{'title': '[정품] 디올 레이디백 미듐', 'url': 'https://cafe.naver.com/ArticleRead.nhn?clubid=10050146&page=1&menuid=782&boardtype=L&articleid=94

KeyboardInterrupt: 

In [12]:
from kafka import KafkaProducer
import requests
from bs4 import BeautifulSoup as bs
import re
from time import sleep
import json

class Ine(object):
    def __init__(self):
        self.target=["에르메스","루이비통","디올","프라다","구찌","샤넬"]
        self.excepts=["삽니다","매입","구매","구입"]
        self.recentSubject=""
        self.TOPIC_NAME="test"
        self.BASE_URL="https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.menuid=782&search.boardtype=L"
        #self.producer=KafkaProducer(bootstrap_servers=brokers)
        #self.brokers=["localhost:9092"]
        
    def clean_text(self,text):
            cleaned_text = re.sub('[a-zA-Z]','',text)
            cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]', '', cleaned_text)
            return cleaned_text

    def data_collect(self):

        with requests.Session() as s:
            #BASE_URL="https://cafe.naver.com/ArticleList.nhn?search.clubid=10050146&search.menuid=782&search.boardtype=L"
            res=s.get(self.BASE_URL)
            if res.status_code==requests.codes.ok:
                par_soup=bs(res.text, 'html.parser')
                number=par_soup.select_one("#main-area > div:nth-child(4) > table > tbody > tr:nth-child(1) > td.td_article > div.board-number > div").text

                if self.recentSubject==number:
                    print('there is no new product')
                    
                    sleep(3)
                    pass
                elif self.recentSubject !=number:
                    self.recentSubject=number
                    print(self.recentSubject,"new number -> start preprocessing")
                    sleep(2)
                    tr_arr=par_soup.select_one("#main-area > div:nth-child(4) > table > tbody > tr")
                    a_tag=tr_arr.select_one('td.td_article > div.board-list > div > a')
                    map = {
                    "title":a_tag.text.strip(),
                    "url":'https://cafe.naver.com' + a_tag["href"],
                    "brand": " "
                    }
                    #print(map)
                    #sleep(2)
                if any(x in map["title"] for x in self.target) and not any(x in map["title"] for x in self.excepts):
                    msg_split=self.clean_text(map["title"]).split(" ")
                    for msg in msg_split:
                        for tar in self.target:
                            val = msg.find(tar)
                            if val !=-1:
                                map['brand']=tar
                                print("intermediate_stage.py로 데이터 전송, index:", val)
                                print(map)
#                                sleep(3)
                    


# while True:
#     vo = Ine()
#     #print(vo.data_collect())
#     print(vo.recentSubject)
#     sleep(3)
#  #       print(map)
#         #brokers=["localhost:9092"]
#         #producer = KafkaProducer(bootstrap_servers=brokers)
        
#         #producer.send(vo.TOPIC_NAME, json.dumps(map).encode("utf-8"))
#         #producer.flush()
#         #print(map)



In [13]:
while True:
    vo=Ine()
    print(vo.data_collect())

945555161 new number -> start preprocessing
None
945555161 new number -> start preprocessing
None
945555208 new number -> start preprocessing
None
945555208 new number -> start preprocessing
None
945555208 new number -> start preprocessing


KeyboardInterrupt: 